In [1]:
import numpy as np

import random
import h5py
from keras import backend as K
from nn_util import *
from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras.models import load_model
from keras.losses import *
import multiprocessing
from sklearn.cluster import KMeans, MiniBatchKMeans

import os
import random
import time
import matplotlib
import matplotlib.pyplot as plt
import gc
import psutil
import math

# for reproducibility
np.random.seed(1337) 
random.seed(1337)

np.set_printoptions(formatter={'float_kind':'{:4f}'.format})

Using TensorFlow backend.


In [2]:
# control amount of GPU memory used
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto()
config.gpu_options.allow_growth=True
set_session(tf.Session(config=config))

In [3]:
# external custom code I wrote
from load_data import *
from windowing import *
from pesq import *
from consts import *
from nn_blocks import *
from perceptual_loss import *
from evaluation import *

In [4]:
[train_paths, val_paths, test_paths], \
[train_waveforms, val_waveforms, test_waveforms], \
[train_procwave, val_procwave, test_procwave], \
[train_wparams, val_wparams, test_wparams], \
[train_windows, val_windows, test_windows] = load_data(TRAIN_SIZE, VAL_SIZE, TEST_SIZE)

In [5]:
# flatten all of the train windows into vectors
train_processed = np.array([i for z in train_windows for i in z])
train_processed = np.reshape(train_processed, (train_processed.shape[0], WINDOW_SIZE,))

# randomly shuffle data, if we want to
if (RANDOM_SHUFFLE):
    train_processed = np.random.permutation(train_processed)
    
print train_processed.shape
print np.mean(train_processed, axis=None)
print np.std(train_processed, axis=None)
print np.min(train_processed, axis = None)
print np.max(train_processed, axis = None)

(101814, 512)
2.44142e-06
0.0994002
-1.0
1.0


In [6]:
CHANNEL_SIZE = WINDOW_SIZE / 2

# ---------------------------------------------------------------------------
# autoencoder: takes an audio window, compresses it, and tries to reconstruct it
# ---------------------------------------------------------------------------
def autoencoder_structure():   
    # - - - - - - - - - - - - - - - - - - - - -
    # parameters
    # - - - - - - - - - - - - - - - - - - - - -   
    NCHAN = 64
    FILT_SIZE = 9
    
    # feature extractor module, used in both encoder and decoder
    #     (structure is the same, but weights aren't shared)
    def feature_extractor():
        def f(inp):
            out = inp
            
            out = residual_block(NCHAN, FILT_SIZE, 1)(out)
            out = residual_block(NCHAN, FILT_SIZE, 2)(out)
            out = residual_block(NCHAN, FILT_SIZE, 4)(out)
            out = residual_block(NCHAN, FILT_SIZE, 8)(out)
            
            return out
        
        return f

    # - - - - - - - - - - - - - - - - - - - - -
    # encoder
    # - - - - - - - - - - - - - - - - - - - - -
    enc_input = Input(shape = (WINDOW_SIZE,))
    enc = Reshape((WINDOW_SIZE, 1))(enc_input)
    
    # processing steps
    enc = channel_change_block(NCHAN, FILT_SIZE)(enc)  
    enc = feature_extractor()(enc)
    enc = downsample_block(NCHAN, FILT_SIZE)(enc)
    enc = feature_extractor()(enc)
    enc = channel_change_block(1, FILT_SIZE)(enc)
    
    # quantization (real numbers => soft bin assignments)
    enc = SoftmaxQuantization()(enc)
    
    enc = Model(inputs = enc_input, outputs = enc, name = 'encoder')
    
    # - - - - - - - - - - - - - - - - - - - - -
    # decoder
    # - - - - - - - - - - - - - - - - - - - - -
    dec_input = Input(shape = (CHANNEL_SIZE, NBINS))
    dec = dec_input
    
    # "dequantization" (soft bin assignments => real numbers)
    dec = SoftmaxDequantization()(dec)
    
    # processing steps
    dec = channel_change_block(NCHAN, FILT_SIZE)(dec)
    dec = feature_extractor()(dec)
    dec = upsample_block(NCHAN, FILT_SIZE)(dec)
    dec = feature_extractor()(dec)
    dec = channel_change_block(1, FILT_SIZE)(dec)
    
    dec = Reshape((WINDOW_SIZE,))(dec)
    dec = Model(inputs = dec_input, outputs = dec, name = 'decoder')
    
    # return both encoder and decoder
    return enc, dec

In [7]:
# map for load_model
KERAS_LOAD_MAP = {'PhaseShiftUp1D' : PhaseShiftUp1D,
                  'code_entropy' : code_entropy,
                  'code_sparsity' : code_sparsity,
                  'rmse' : rmse,
                  'SoftmaxQuantization' : SoftmaxQuantization,
                  'SoftmaxDequantization' : SoftmaxDequantization,
                  'DFT_REAL' : DFT_REAL,
                  'DFT_IMAG' : DFT_IMAG,
                  'MEL_FILTERBANKS' : MEL_FILTERBANKS,
                  'keras_dft_mag' : keras_dft_mag,
                  'keras_dct' : keras_dct,
                  'perceptual_transform' : perceptual_transform,
                  'perceptual_distance' : perceptual_distance}

In [8]:
# construct autoencoder
ac_input = Input(shape = (WINDOW_SIZE,))

encoder, decoder = autoencoder_structure()
ac_reconstructed = decoder(encoder(ac_input))
autoencoder = Model(inputs = [ac_input], outputs = [ac_reconstructed])

In [9]:
# model parameters
loss_weights = [30.0, 5.0, 10.0, 1.0]
loss_functions = [rmse, perceptual_distance, code_sparsity, code_entropy]
n_recons = 2
n_code = 2
assert(n_recons + n_code == len(loss_weights))
assert(len(loss_weights) == len(loss_functions))

In [10]:
# model specification
model_input = Input(shape = (WINDOW_SIZE,))
model_embedding = encoder(model_input)
model_reconstructed = decoder(model_embedding)

model = Model(inputs = [model_input], outputs = [model_reconstructed] * n_recons + \
                                            [model_embedding] * n_code)

/usr/local/lib/python2.7/dist-packages/keras/engine/topology.py:1511: UserWarning: The list of outputs passed to the model is redundant. All outputs should only appear once. Found: [<tf.Tensor 'decoder_1/reshape_2/Reshape:0' shape=(?, 512) dtype=float32>, <tf.Tensor 'decoder_1/reshape_2/Reshape:0' shape=(?, 512) dtype=float32>, <tf.Tensor 'encoder_1/softmax_quantization_1/cond/Merge:0' shape=(?, 256, 32) dtype=float32>, <tf.Tensor 'encoder_1/softmax_quantization_1/cond/Merge:0' shape=(?, 256, 32) dtype=float32>]
  ' Found: ' + str(self.outputs))


In [11]:
model.compile(loss = loss_functions,
              loss_weights = loss_weights,
              optimizer = Adam())

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, 512)               0         
_________________________________________________________________
encoder (Model)              (None, 256, 32)           742447    
_________________________________________________________________
decoder (Model)              (None, 512)               816399    
Total params: 1,558,846
Trainable params: 1,558,846
Non-trainable params: 0
_________________________________________________________________


In [12]:
# model without any training
test_model_on_wav("./SA1.wav", "", autoencoder,
                  save_recons = False)

MSE:         154586.0
Avg err:     210.205
PESQ:        1.05026233196


[154586.23, 210.20505, 1.0502623319625854]

In [13]:
# saves current model
def save_model(prefix = 'best'):
    os.system('rm ./' + prefix + '_model.h5')
    os.system('rm ./' + prefix + '_auto.h5')
    #os.system('rm ./' + prefix + '_quant_bins.npy')
    
    model.save('./' + prefix + '_model.h5')
    autoencoder.save('./' + prefix + '_auto.h5')
    #np.save('./' + prefix + '_quant_bins.npy', K.eval(QUANT_BINS))
    
    f = h5py.File('./' + prefix + '_model.h5', 'r+')
    del f['optimizer_weights']
    f.close()

In [14]:
def evaluate_training(autoencoder, lead = ""):
    def set_evaluation(paths, eval_idxs):
        before_after_pairs = [run_model_on_wav(paths[i],
                                               autoencoder,
                                               argmax = True)
                              for i in eval_idxs]
        
        def thread_func(my_id, q):
            my_slice = np.arange(my_id, len(eval_idxs), NUM_THREADS)
            
            for i in my_slice:
                p = before_after_pairs[i]
                q.put(evaluation_metrics(p[0], p[1]))
        
        q = multiprocessing.Queue()
        threads = [multiprocessing.Process(target = thread_func,
                                           args = (i, q))
                   for i in xrange(0, NUM_THREADS)]
        [t.start() for t in threads]
        [t.join() for t in threads]
        results = np.array([q.get() for i in xrange(0, len(eval_idxs))])
        
        return results
    
    train_eval_idxs = random.sample(range(0, len(train_paths)), TRAIN_EVALUATE)
    val_eval_idxs = random.sample(range(0, len(val_paths)), VAL_EVALUATE)
    
    print lead + "Format: [MSE, avg err, PESQ]"
    
    # train set evaluation
    train_metrics = set_evaluation(train_paths,
                                   train_eval_idxs)
    print lead + "    Train: (mean)", np.mean(train_metrics, axis = 0)
    print lead + "    Train: (max) ", np.max(train_metrics, axis = 0)
    print lead + "    Train: (min) ", np.min(train_metrics, axis = 0)
    
    # validation set evaluation
    val_metrics = set_evaluation(val_paths,
                                 val_eval_idxs)
    print lead + "    Val:   (mean)", np.mean(val_metrics, axis = 0)
    print lead + "    Val:   (max) ", np.max(val_metrics, axis = 0)
    print lead + "    Val:   (min) ", np.min(val_metrics, axis = 0)
    
    # returns mean PESQ on validation
    return np.mean(val_metrics, axis = 0)[2]

In [15]:
X_train = np.copy(train_processed)
ntrain = X_train.shape[0]

NUM_EPOCHS = 300
EPOCHS_BEFORE_QUANT_ON = 5

ORIG_BITRATE = 256.00
TARGET_BITRATE = 20.00
PRE_ENTROPY_RATE = ORIG_BITRATE * (float(CHANNEL_SIZE) / WINDOW_SIZE)

TARGET_ENTROPY = (TARGET_BITRATE / PRE_ENTROPY_RATE * 16.0)
TARGET_ENTROPY *= (STEP_SIZE / float(WINDOW_SIZE))
TARGET_ENTROPY_FUZZ = 0.1

TAU_CHANGE_RATE = 0.025
INITIAL_TAU = 0.5

NUM_QUANT_VECS = 5000

STARTING_LR = 0.00025
ENDING_LR = 0.0001

print "Target entropy:", TARGET_ENTROPY

Target entropy: 2.34375


In [16]:
best_val_pesq = 0.0
K.set_value(tau, 0.0)
T_i = 0.0
K.set_value(QUANTIZATION_ON, False)

In [17]:
np.set_printoptions(formatter={'float_kind':'{:4f}'.format})
lead = "    "

for epoch in range(1, NUM_EPOCHS + 1):
    print "Epoch " + str(epoch) + ":"

    # present batches randomly each epoch
    lis = range(0, ntrain, BATCH_SIZE)
    random.shuffle(lis)
    num_batches = len(lis)
    
    # keep track of start time and current batch #
    i = 0
    startTime = time.time()
    for idx in lis:
        # cosine annealing for model's learning rate
        train_pct = T_i / float(NUM_EPOCHS)
        opt_lr = ENDING_LR + 0.5 * (STARTING_LR - ENDING_LR) * (1 + math.cos(3.14159 * train_pct))
        T_i += (1.0 / num_batches)
        K.set_value(model.optimizer.lr, opt_lr)
        
        batch = X_train[idx:idx+BATCH_SIZE, :]
        nbatch = batch.shape[0]
               
        # train autoencoder
        a_y = [batch] * n_recons + \
              [np.zeros((nbatch, 1, 1))] * n_code

        a_losses = model.train_on_batch(batch, a_y)
        
        # print statistics every 10 batches so we know what's going on
        if (i % 10 == 0):
            printStr = "        \r" + lead + str(i * BATCH_SIZE) + ": "
            print printStr,
            
            loss_arr = np.asarray(a_losses)
            print loss_arr,
            
            if (len(loss_weights) > 1 and len(loss_arr) > 1):
                for w in xrange(0, len(loss_weights)):
                    loss_arr[w + 1] *= loss_weights[w]
                print loss_arr,
            
            print K.get_value(tau), opt_lr,
        
        i += 1
    print ""
    
    # print elapsed time for epoch
    elapsed = time.time() - startTime
    print lead + "Total time for epoch: " + str(elapsed) + "s"
    
    # ---------------------------------------------------------
    # estimate code entropy from random samples (if quantization is on)
    # ---------------------------------------------------------
    if (K.get_value(QUANTIZATION_ON) > 0):
        NUM = 20000
        rows = np.random.randint(X_train.shape[0], size = NUM)
        to_predict = np.copy(X_train[rows, :])
        code = encoder.predict(to_predict, verbose = 0, batch_size = 128)
        
        all_onehots = np.reshape(code, (-1, NBINS))
        onehot_hist = np.sum(all_onehots, axis = 0)
        onehot_hist /= np.sum(onehot_hist)

        entropy = 0
        for i in onehot_hist:
            if (i < 1e-5): continue
            entropy += i * math.log(i, 2)
        entropy = -entropy

        print lead + "----------------"
        print lead + "Code entropy:", entropy

        # ---------------------------------------------------------
        # handle updating entropy weight (tau)
        # ---------------------------------------------------------
        old_tau = K.get_value(tau)

        if (entropy < TARGET_ENTROPY - TARGET_ENTROPY_FUZZ):
            new_tau = old_tau - TAU_CHANGE_RATE
            if (new_tau < 0.0):
                new_tau = 0.0

            K.set_value(tau, new_tau)
            print lead + "Updated tau from", old_tau, "to", new_tau
        elif (entropy > TARGET_ENTROPY + TARGET_ENTROPY_FUZZ):
            new_tau = old_tau + TAU_CHANGE_RATE

            K.set_value(tau, new_tau)
            print lead + "Updated tau from", old_tau, "to", new_tau
        else:
            print lead + "Tau stays at", old_tau
    
    # ---------------------------------------------------------
    # evaluate autoencoder on training/validation data evey epoch
    # ---------------------------------------------------------
    startTime = time.time()
    print lead + "----------------"
    print lead + "Evaluating autoencoder..."
    
    
    metrics = test_model_on_wav("./SA1.wav", "./train_output/SA1_train_epoch" + str(epoch),
                                autoencoder, lead = lead, verbose = False, argmax = False)
    print lead + "SA1:         ", metrics
    if (K.get_value(QUANTIZATION_ON) > 0):
        metrics = test_model_on_wav("./SA1.wav", "./train_output/SA1_train_epoch" + str(epoch),
                                    autoencoder, lead = lead, verbose = False, argmax = True)
        print lead + "SA1 (arg):   ", metrics
    
    metrics = test_model_on_wav("./SX383.wav", "./train_output/SX383_train_epoch" + str(epoch),
                                autoencoder, lead = lead, verbose = False, argmax = False)
    print lead + "SX383:       ", metrics
    if (K.get_value(QUANTIZATION_ON) > 0):
        metrics = test_model_on_wav("./SX383.wav", "./train_output/SX383_train_epoch" + str(epoch),
                                    autoencoder, lead = lead, verbose = False, argmax = True)
        print lead + "SX383 (arg): ", metrics
    
    if (K.get_value(QUANTIZATION_ON) > 0):
        val_pesq = evaluate_training(autoencoder, lead)
        if (val_pesq > best_val_pesq and entropy <= TARGET_ENTROPY):
            print lead + "NEW best model! Validation mean-PESQ", val_pesq

            print lead + "Saving model..."
            save_model()
            best_val_pesq = val_pesq
            patience_epoch = epoch
        else:
            print lead + "Best validation mean-PESQ seen:", best_val_pesq
    
    elapsed = time.time() - startTime
    print lead + "Total time for evaluation: " + str(elapsed) + "s"
    
    gc.collect()
    process = psutil.Process(os.getpid())
    mem_used = process.memory_info().rss
    print lead + "Total memory usage: " + str(mem_used)
    
    # ---------------------------------------------------------
    # turn quantization on after a certain # of epochs
    # ---------------------------------------------------------
    if (epoch == EPOCHS_BEFORE_QUANT_ON):
        print lead + "----------------"
        print lead + "Turning quantization on!"
        
        random_windows = []
        for i in xrange(0, NUM_QUANT_VECS):
            w_idx = random.randint(0, train_processed.shape[0] - 1)
            random_windows.append(train_processed[w_idx])

        random_windows = np.array(random_windows)
        print lead + "    Selecting random code vectors for analysis..."
        encoded_windows = encoder.predict(random_windows, batch_size = 128, verbose = 0)
        encoded_windows = encoded_windows[:, :, 0]
        encoded_windows = np.reshape(encoded_windows, (-1, 1))

        print lead + "    K means clustering for bins initialization..."
        km = MiniBatchKMeans(n_clusters = NBINS).fit(encoded_windows)
        clustered_bins = km.cluster_centers_.flatten()
        
        cluster_score = np.sqrt(np.median(np.min(km.transform(encoded_windows), axis = 1)))
        print lead + "    Done. Cluster score:", cluster_score
        
        K.set_value(QUANTIZATION_ON, True)
        K.set_value(QUANT_BINS, clustered_bins)
        K.set_value(tau, INITIAL_TAU)

Epoch 1:
    101120:  [2.837456 0.007795 0.520719 0.000000 0.000000] [2.837456 0.233863 2.603593 0.000000 0.000000] 0.0 0.0002499959494691                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               
    Total time for epoch: 150.688800097s
    ----------------
    Evaluating autoencoder...
    SA1:          [2619.2056, 27.280355, 4.057804584503174]
    SX

    101120:  [5.028639 0.013861 0.547560 0.056788 1.307121] [5.028639 0.415840 2.737801 0.567877 1.307121] 0.525 0.000249799019475                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          
    Total time for epoch: 146.441956997s
    ----------------
    Code entropy: 2.52164323553
    Updated tau from 0.525 to 0.549999976158
    ----------------
    Evaluat

        Train: (mean) [8573.048562 47.276345 3.801522]
        Train: (max)  [41977.738281 104.282631 4.340391]
        Train: (min)  [952.619507 17.715698 2.980741]
        Val:   (mean) [9942.449518 49.350551 4.001278]
        Val:   (max)  [77666.578125 115.390823 4.358169]
        Val:   (min)  [433.513977 11.739481 2.960002]
    Best validation mean-PESQ seen: 3.79081161737
    Total time for evaluation: 13.5308301449s
    Total memory usage: 3799007232
Epoch 12:
    101120:  [4.525342 0.015253 0.488417 0.040299 1.222669] [4.525342 0.457597 2.442085 0.402991 1.222669] 0.525 0.000249409345347                                                                                                                                                                                                                                                                                                                                                                                                             

    101120:  [4.372769 0.012986 0.456113 0.036957 1.333046] [4.372769 0.389589 2.280564 0.369571 1.333046] 0.55 0.000248950691604                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          
    Total time for epoch: 143.657873869s
    ----------------
    Code entropy: 2.46075237504
    Updated tau from 0.55 to 0.574999952316
    ----------------
    Evaluatin

        Train: (mean) [9336.960269 48.771513 3.835536]
        Train: (max)  [50911.812500 133.965210 4.358758]
        Train: (min)  [1003.418701 18.396082 2.106988]
        Val:   (mean) [8664.497533 46.596063 4.078061]
        Val:   (max)  [64197.097656 104.186989 4.388891]
        Val:   (min)  [378.519379 11.079936 3.525862]
    Best validation mean-PESQ seen: 4.01678643465
    Total time for evaluation: 13.7584750652s
    Total memory usage: 3800940544
Epoch 21:
    101120:  [4.332998 0.013232 0.470337 0.032136 1.263002] [4.332998 0.396946 2.351686 0.321365 1.263002] 0.55 0.000248195051381                                                                                                                                                                                                                                                                                                                                                                                                             

...

        Train: (mean) [6623.566746 40.288434 4.170333]
        Train: (max)  [62989.699219 104.194389 4.495220]
        Train: (min)  [775.817993 14.779306 3.315813]
        Val:   (mean) [7088.211617 41.619697 4.265116]
        Val:   (max)  [49435.765625 92.117439 4.496298]
        Val:   (min)  [307.237000 9.955906 3.503603]
    Best validation mean-PESQ seen: 4.27203153372
    Total time for evaluation: 14.8441059589s
    Total memory usage: 3799605248
Epoch 282:
    101120:  [3.172190 0.009703 0.317909 0.016759 1.123951] [3.172190 0.291103 1.589545 0.167591 1.123951] 0.5 0.000101329600807                                                                                                                                                                                                                                                                                                                                                                                                                

    101120:  [3.187383 0.010114 0.315716 0.016695 1.138441] [3.187383 0.303418 1.578578 0.166945 1.138441] 0.5 0.000100805467814                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          
    Total time for epoch: 158.739129066s
    ----------------
    Code entropy: 2.3323615646
    Tau stays at 0.5
    ----------------
    Evaluating autoencoder...
    SA1:

        Train: (mean) [6548.384418 41.526287 4.142732]
        Train: (max)  [45318.187500 117.467018 4.479138]
        Train: (min)  [425.738342 13.662574 3.083311]
        Val:   (mean) [7032.418399 41.830808 4.251340]
        Val:   (max)  [48260.082031 93.776611 4.469503]
        Val:   (min)  [313.522705 10.012604 3.426009]
    Best validation mean-PESQ seen: 4.27203153372
    Total time for evaluation: 14.6602210999s
    Total memory usage: 3800993792
Epoch 291:
    101120:  [3.184126 0.010113 0.310924 0.016644 1.159671] [3.184126 0.303398 1.554619 0.166438 1.159671] 0.5 0.000100333428199                                                                                                                                                                                                                                                                                                                                                                                                               

    101120:  [3.098951 0.010105 0.296567 0.016593 1.147040] [3.098951 0.303145 1.482833 0.165933 1.147040] 0.5 0.000100103105218                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          
    Total time for epoch: 158.795071125s
    ----------------
    Code entropy: 2.31704733757
    Tau stays at 0.5
    ----------------
    Evaluating autoencoder...
    SA1

        Train: (mean) [6603.668606 41.097459 4.144648]
        Train: (max)  [33890.324219 117.633522 4.452605]
        Train: (min)  [363.144623 11.974672 2.897854]
        Val:   (mean) [6995.582542 41.563843 4.254646]
        Val:   (max)  [48252.410156 93.361107 4.486503]
        Val:   (min)  [304.980194 9.905242 3.215588]
    Best validation mean-PESQ seen: 4.27203153372
    Total time for evaluation: 15.0168988705s
    Total memory usage: 3800985600
Epoch 300:
    101120:  [3.216215 0.010013 0.315677 0.016933 1.168113] [3.216215 0.300389 1.578387 0.169325 1.168113] 0.5 0.00010000000025                                                                                                                                                                                                                                                                                                                                                                                                                 

In [ ]:
save_model('end')